In [1]:
import Atlas
import pandas as pd

#### Initialize market variables

Atlas has tape-based AD, so later we will use the tape.

In [2]:
tape = Atlas.Tape()

In [3]:
evalDate = Atlas.Date(1, Atlas.August, 2020)
store = Atlas.MarketStore(evalDate, Atlas.CLP()) # store with CLP as base currency

# define curve
curveRate = Atlas.dual(0.03)
tape.registerInput(curveRate) # we need to register the input to the tape for later use
tape.newRecording() # start recording, for later use

curveDayCounter = Atlas.Actual360()
curveCompounding = Atlas.Simple
curveFrequency = Atlas.Annual
strategy = Atlas.FlatForwardStrategy(evalDate, curveRate, curveDayCounter, curveCompounding, curveFrequency)
curve = Atlas.YieldTermStructure(strategy)

# define rate index
index = Atlas.RateIndex(evalDate, curveFrequency, curveDayCounter, curveFrequency, curveCompounding)

# add curve
store.addCurve("ExampleCurve", curve, index)


#### Create an instrument

We initialize the instrument with the corresponding discount curve id (passing the context parameter). As default, instrument coupon's are set as "local" currency, meaning that their current currency will match the store local currency.

In [4]:
#define interest rate
rateValue = Atlas.dual(0.05)
dayCounter = Atlas.Thirty360()
compounding = Atlas.Simple
frequency = Atlas.Annual

rate = Atlas.InterestRate(rateValue, dayCounter, compounding, frequency)
discountContext = store.curveContext("ExampleCurve")
# define zero coupon instrument
notional = 100
startDate = evalDate
endDate = Atlas.Date(1, Atlas.August, 2025)
paymentFrequency = Atlas.Semiannual
instrument = Atlas.FixedRateBulletInstrument(startDate, endDate, paymentFrequency, notional, rate, discountContext)

#### Inspect the cashflows
We can use the CashflowProfiler visitor to check the instrument cashflows.

In [5]:
profiler = Atlas.CashflowProfiler()
profiler.visit(instrument)
interest = profiler.interests()
redemptions = profiler.redemptions()

df = pd.DataFrame({"Interest": interest, "Redemptions": redemptions})
df

,Interest,Redemptions
2021-2-1,2.5,NaN
2021-8-1,2.5,NaN
2022-2-1,2.5,NaN
2022-8-1,2.5,NaN
2023-2-1,2.5,NaN
2023-8-1,2.5,NaN
2024-2-1,2.5,NaN
2024-8-1,2.5,NaN
2025-2-1,2.5,NaN
2025-8-1,2.5,NaN


### Evaluation process

#### 1. Index the instrument
In the indexing phase, market variables are obtained and stored in a MarketRequest object. This information will be used by a Model, which is in charge of producing all the market data needed for later calculations.

In [6]:
indexer = Atlas.Indexer()
indexer.visit(instrument)
request = indexer.request()

#### 2. Setup a model and simulate market variables
Currently the only model available (SpotMarketDataModel), takes the market information and generates values assuming common linear product's assumptions.

In [7]:
model = Atlas.SpotMarketDataModel(request, store)
marketData = model.marketData(evalDate)

#### 3. Setup a visitor and evaluate
Visitor are the ones in charge to do evaluations. When visiting, Visitor will execute the precise code needed for each type of instrument.

- Instrument NPV: ```NPVCalculator```

This visitor calculates the NPV of each instruments and adds it to an internal variables called npv_, so if it visits many instrument, the value returned by ```results``` will be the sum of each NPV. In the case of a fixed bond, the NPV is being calculated as:

$$NPV^l = \frac{\Sigma_{1}^{N}c_{i}^{f}df^{f}_{i}}{fx^{f/l}}$$

In [8]:
npv = Atlas.dual(0.0)
tape.registerOutput(npv)

npvCalculator = Atlas.NPVCalculator(marketData)
npvCalculator.visit(instrument)
npv = npvCalculator.results()
print("NPV: {:.4f}".format(Atlas.getValue(npv)))

NPV: 109.8990


If we want to calculate the insturment duration, we can use the tape (AD). In this case, the duration is being calculated as:

$$Dur = \frac{dNPV}{dr}$$

In [9]:
npv.setDerivative(-0.0001)
tape.computeAdjoints()
print("Derivative: {:.4f}".format(curveRate.getDerivative()*100))

Derivative: 4.3928


- Fixed Income Par Rate: ```ParSolver```

This visitor calculates the par rate of a given instrument (in this case, rates are not "accumulated" as before). The par rate is calculated, for a fixed rate instruments as follows:

$$r = \argmin_r (\frac{\Sigma_{1}^{T}c_{i}(r) df_{i}}{N} - df_0)^2$$

Where $df_0$ helps bringing the disbursement to the current evaluation date.

In [10]:
parSolver = Atlas.ParSolver(marketData)
parSolver.visit(instrument)
rate = parSolver.results()
print("Par Rate: {:.4f}%".format(Atlas.getValue(rate)*100))

Par Rate: 2.8579%


- Fixed Income Z-Spread: ```ZSpreadCalculator```

This visitor calculates the z-spread of a fixed rate instrument (does not apply to other types of instruments). 

$$s = \argmin_s ({\Sigma_{1}^{N}c_{i} df_{i}(s)} - NPV_{target})^2$$

Where $df_i$ will be calculated using the given day counter, compounding and frequency.

In [12]:
targetNPV = Atlas.dual(100)
zspreadCalculator = Atlas.ZSpreadCalculator(marketData, targetNPV, curveDayCounter, curveCompounding, curveFrequency)
zspreadCalculator.visit(instrument)
zspread = zspreadCalculator.results()
print("Z-spread: Curve+{:.4f} bps".format(Atlas.getValue(zspread)*10000))

Z-spread: Curve+249.0599 bps
